In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
URL = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N"
response = requests.get(URL)
soup = BeautifulSoup(response.text)
print(soup.prettify())

In [6]:
member_divs = soup.findAll("div", attrs={'class' : 'Members--list-item'})

In [74]:
len(member_divs)

4

In [80]:
member_divs = soup.findAll("div", attrs={'class' : 'Members--list-item'})
candidate_text = member_divs[0].find("a").text
affiliation = re.findall(r"\([A-Z]\)", candidate_text)[0][1]
affiliation
# t = member_divs[1].find("h2").find("strong").text.strip()
# candidate_info = re.findall(r"[^\n\t+]+", t)
# affiliation = re.findall(r"\([A-Z]\)", candidate_info[0])[0][1]
# name = re.sub(r"\([A-Z]\)", "", candidate_info[0]).strip()
# name

# rr = re.findall(r"\([A-Z]\)", t)[0]
# rr[1]
# re.findall(r"Incumbent", t)
# t=re.sub(r"\([A-Z]\)","",t)
# t= re.sub(r"Incumbent", "", t)
# t = re.sub("[^0-9a-zA-Z\s]+", "", t)
# t.strip()
# t

'R'

In [108]:
def get_candidate_info(member_div):
    
    results = {"Name": "", "Affiliation": "", "Incumbent": 0, "Winner": 0, "Vote %": ""}
    if member_div.find("a") != None:
        candidate_text = member_div.find("a").text
        affiliation = re.findall(r"\([A-Z]\)", candidate_text)[0][1]
        incumbent = 1 if "Incumbent" in candidate_text else 0
        results["Incumbent"] = incumbent
        candidate_text=re.sub(r"\([A-Z]\)","",candidate_text)
        candidate_text= re.sub(r"Incumbent", "", candidate_text)
        name = re.sub("[^0-9a-zA-Z\s]+", "", candidate_text)

    else:
        txt = member_div.find("h2").find("strong").text.strip()
        candidate_info = re.findall(r"[^\n\t+]+", txt)
        affiliation = re.findall(r"\([A-Z]\)", candidate_info[0])[0][1]
        name = re.sub(r"\([A-Z]\)", "", candidate_info[0]).strip()
    if member_div.find("span", attrs={'class' : 'winner'}):
        results["Winner"] = 1
    votepct = member_div.find("span", attrs={'class' : 'Members--vote-pct'}).text if member_div.find("span", attrs={'class' : 'Members--vote-pct'}) else ""
    re.findall(r"\d+\.\d+%", votepct)
    results["Name"] = name
    results["Affiliation"] = affiliation
        
    return results



In [94]:
URL

'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

In [99]:
state_districtnum = re.findall(r'[A-Z][A-Z]\d+', URL)[0]
state = state_districtnum[:2]
districtnum = re.findall(r'\d+',state_districtnum)[0]
districtnum

'07'

In [112]:
get_candidate_info(member_divs[3])

{'Name': 'Scott Vieira Jr', 'Affiliation': 'I', 'Incumbent': 0, 'Winner': 0}

In [126]:
votepct = member_divs[3].find("span", attrs={'class' : 'Members--vote-pct'}).text
re.findall(r"\d+\.\d+%", votepct)


['0.6%']

In [121]:
votepct

'(27.3% of vote)'